# ConvNet on MNIST

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

import sys
sys.path.append('../')
from Feedforward import *

## 1. Load MNIST

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
batch_X, batch_y = mnist.train.next_batch(100)

In [4]:
print(np.reshape(batch_X, [-1,28,28,1]).shape)
print(batch_y.shape)


(100, 28, 28, 1)
(100, 10)


## 2. Model definition

In [5]:
sess = tf.InteractiveSession()

In [6]:
input_size= [28, 28]
n_outputs=  batch_y.shape[1]
n_input_maps= 1

n_filters = [32, 64]   
filter_size = [[5,5], [5,5]] 
pool_size =   [[2,2], [2,2]] 
n_hidden = [1024]

batch_size= 100

In [7]:
# for dropout
drop_prob = tf.placeholder(tf.float32)

# Define ConvNet
conv_net= ConvNet( input_size, n_input_maps, n_outputs, n_filters, filter_size, pool_size, n_hidden, name='ConvNet')

# Define train setup
train= conv_net.setup(batch_size, drop_prob, loss_type= 'cross_entropy')

# Optimizer.
optimizer = tf.train.AdamOptimizer(1e-3).minimize(train.loss) #0.001
  
# Predictions for the training, validation, and test data.
train_prediction = tf.nn.softmax(train.y)

Shape of input matrix entering to Fully connected layers: [100, 4, 4, 64]


## 3. Training

In [8]:
def accuracy(predictions, labels):
    if n_outputs == 1:
        return (100.0 * np.sum(np.greater(predictions, 0.5) == np.greater(labels, 0.5))/ predictions.shape[0])
    else:
        return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

In [9]:
num_steps = 5000
n_logging= 100
n_test_logg= 10 # number of evaluations on test dataset (for logging information)

tf.initialize_all_variables().run()
print('Initialized')

mean_loss= 0
train_accuracy= 0
for step in range(num_steps):
    batch_X, batch_y= mnist.train.next_batch(batch_size)
    batch_X= np.reshape(batch_X, [-1,28,28,1])
    
    
    feed_dict = {train.inputs : batch_X, train.labels : batch_y, drop_prob:1.0}
    
    _, l, train_pred = sess.run([optimizer, train.loss, train_prediction],feed_dict=feed_dict)
    
    train_accuracy += accuracy(train_pred, batch_y)
    mean_loss += l    
    
    if step%n_logging == 0:
        # train
        train_accuracy= train_accuracy/n_logging
        
        # test
        test_accuracy= 0
        for i in range(n_test_logg):
            batch_X_test, batch_y_test= mnist.test.next_batch(batch_size) 
            batch_X_test= np.reshape(batch_X_test, [-1,28,28,1])

            pred = train_prediction.eval(feed_dict={train.inputs: batch_X_test, drop_prob: 1.0})
            
            test_accuracy += accuracy(pred, batch_y_test)
        test_accuracy= test_accuracy/n_test_logg
            
        print(step, ', train:',train_accuracy,' | test:', test_accuracy, ' | loss:', mean_loss/n_logging)
        mean_loss= 0
        train_accuracy= 0

Initialized
0 , train: 0.1  | test: 21.6  | loss: 0.0364383888245
100 , train: 86.93  | test: 94.2  | loss: 0.462256414592
200 , train: 96.71  | test: 95.9  | loss: 0.105317889675
300 , train: 96.86  | test: 97.2  | loss: 0.100662346762
400 , train: 97.69  | test: 96.7  | loss: 0.0753901388682
500 , train: 97.66  | test: 99.6  | loss: 0.0725157791423
600 , train: 98.73  | test: 98.8  | loss: 0.0400344116311
700 , train: 98.9  | test: 99.8  | loss: 0.0403927720827
800 , train: 98.69  | test: 99.4  | loss: 0.0422864675848
900 , train: 98.76  | test: 99.2  | loss: 0.0384863246977
1000 , train: 98.91  | test: 99.4  | loss: 0.0359039069305
1100 , train: 98.65  | test: 98.9  | loss: 0.0413813458616
1200 , train: 99.13  | test: 98.6  | loss: 0.027356455992
1300 , train: 99.2  | test: 99.2  | loss: 0.0251433366496
1400 , train: 99.13  | test: 98.9  | loss: 0.02856467203
1500 , train: 99.27  | test: 99.5  | loss: 0.0225193510286
1600 , train: 99.19  | test: 98.5  | loss: 0.0245968748134
1700 , 